<a href="https://colab.research.google.com/github/karankumar211/Native-Language-Identification-Project/blob/main/Task4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# CELL 1: Install Libraries
!pip install numpy scikit-learn librosa
print("Libraries installed.")


In [ ]:
# CELL 2: Mount Drive & Load Data
import numpy as np
import json
from google.colab import drive
import os

print("Mounting Google Drive...")
drive.mount('/content/drive')

save_path = '/content/drive/MyDrive/Colab_Project_Data/'
print("Loading processed Task 1 data from Google Drive...")

try:
    # Load all the .npy files we need
    X_mfcc = np.load(os.path.join(save_path, 'X_mfcc.npy'))
    y_labels = np.load(os.path.join(save_path, 'y_labels.npy'))

    # Load the label mapping
    with open(os.path.join(save_path, 'label_to_int.json'), 'r') as f:
        label_to_int = json.load(f)
    int_to_label = {int(i): name for name, i in label_to_int.items()}

    print("--- Task 1 Data Loaded Successfully ---")
    print(f"MFCC features shape: {X_mfcc.shape}")
    print(f"Labels shape: {y_labels.shape}")
    print(f"Label mapping: {int_to_label}")

except Exception as e:
    print(f"--- ERROR ---")
    print(f"Could not load Task 1 data from Google Drive: {e}")
    print("Please ensure your 'Colab_Project_Data' folder is in 'My Drive'.")
    raise


In [ ]:
# CELL 3: Train Production Model
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

print("Training production model on 100% of the MFCC data...")

# 1. Create and fit the Scaler
# We save this scaler to use on our live audio
scaler = StandardScaler()
X_mfcc_scaled = scaler.fit_transform(X_mfcc)
print("Scaler is fit.")

# 2. Create and fit the SVM model
# This is our final, trained model
model = SVC(kernel='rbf', C=1.0, random_state=42, probability=True)
model.fit(X_mfcc_scaled, y_labels)

print("--- Production Model is Trained and Ready! ---")

In [ ]:
# CELL 4: JavaScript Audio Recorder (Corrected)
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import io
import librosa
import soundfile as sf # Use soundfile for saving

RECORD = """
const sleep = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  // Try to specify a common MIME type
  recorder = new MediaRecorder(stream, { mimeType: 'audio/webm;codecs=opus' })
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.onstop = async e => {
    blob = new Blob(chunks, { type: 'audio/webm' }) // Ensure blob type
    text = await b2text(blob)
    resolve(text)
  }
  recorder.start()
  await sleep(time)
  recorder.stop()
  stream.getTracks().forEach(track => track.stop())
})
"""

def record_audio(raw_filename="recording.webm", final_filename="recording.wav"):
  """
  Records audio, saves it as .webm, then converts and resamples to .wav.
  """
  try:
    print("Click 'Record', allow microphone access, speak, and then click 'Stop'.")
    display(Javascript(RECORD))
    data = eval_js('record(5000)') # Records for 5 seconds

    print("Recording stopped. Processing audio...")
    binary = b64decode(data.split(',')[1])

    # 1. Save the raw recording (likely webm/opus)
    with open(raw_filename, 'wb') as f:
        f.write(binary)
    print(f"Raw recording saved as {raw_filename}")

    # 2. Load with librosa (which uses ffmpeg) to decode and resample
    # This is the step that failed before. Now it's loading from a file.
    wav, sr = librosa.load(raw_filename, sr=16000, mono=True)

    # 3. Save the clean, resampled audio as a standard .wav file
    sf.write(final_filename, wav, 16000)

    print(f"Clean, resampled audio saved as {final_filename}")
    return final_filename
  except Exception as e:
    print(f"An error occurred during recording: {e}")
    # Add a more specific error message for librosa
    if "No audio data found" in str(e):
        print("ERROR: No audio was detected. Please try recording again.")
    return None

In [ ]:
# CELL 5: The Recommendation System
import librosa
import numpy as np

# --- 1. Our Recommendation Dictionary ---
# (You can customize this!)
recommendation_map = {
    "andhra_pradesh": "You might enjoy some spicy Andhra cuisine, like Gongura Mamsam or Pesarattu.",
    "gujrat": "How about some Gujarati specialties? We recommend Dhokla or Thepla.",
    "jharkhand": "We can recommend some local favorites from Jharkhand, like Litti Chokha or Dhuska.",
    "karnataka": "We recommend some classic Karnataka dishes, like Bisi Bele Bath or Mysore Dosa.",
    "kerala": "We detect a Kerala accent. You might enjoy our Appam, Puttu, or Avial! [cite: 29]",
    "tamil": "How about some delicious Tamil food? We recommend Chettinad Chicken or Pongal."
}

# --- 2. The Prediction Function ---
def predict_accent(audio_path):
  """
  Takes an audio file path, processes it, and predicts the accent.
  """
  try:
    print(f"\nAnalyzing accent from {audio_path}...")
    # --- A. Load and Process Audio ---
    waveform, sr = librosa.load(audio_path, sr=None)

    # --- B. Extract MFCCs (same settings as our training) ---
    mfccs = librosa.feature.mfcc(y=waveform, sr=sr, n_mfcc=20)
    mfccs_mean = np.mean(mfccs, axis=1)

    # --- C. Scale Features (CRITICAL STEP) ---
    # We must use the *same scaler* we fit during training
    # We reshape to (1, 20) because the scaler expects a 2D array
    mfccs_scaled = scaler.transform(mfccs_mean.reshape(1, -1))

    # --- D. Make Prediction ---
    prediction_int = model.predict(mfccs_scaled)[0]
    prediction_proba = model.predict_proba(mfccs_scaled)
    confidence = np.max(prediction_proba) * 100

    # --- E. Get Recommendation ---
    accent_name = int_to_label[prediction_int]
    recommendation = recommendation_map.get(accent_name, "No recommendation available.")

    print("\n--- 🍲 ACCENT-AWARE RECOMMENDATION ---")
    print(f"Predicted Accent: {accent_name.replace('_', ' ').title()}")
    print(f"Confidence: {confidence:.2f}%")
    print("\nRecommendation:")
    print(f"> {recommendation}")

  except Exception as e:
    print(f"An error occurred during prediction: {e}")

# --- 3. Run the App ---
audio_file = record_audio()
if audio_file:
  predict_accent(audio_file)

In [ ]:
# CELL 6: Install Gradio
!pip install gradio
import gradio as gr

In [ ]:
# CELL 7: Build and Launch the Gradio Web App (Allowing Uploads)
import gradio as gr
import librosa
import numpy as np

# --- 1. Our Models and Mappings (from previous cells) ---
# (Make sure CELL 2 and CELL 3 have been run)

# --- 2. Our Recommendation Dictionary ---
recommendation_map = {
    "andhra_pradesh": "You might enjoy some spicy Andhra cuisine, like Gongura Mamsam or Pesarattu.",
    "gujrat": "How about some Gujarati specialties? We recommend Dhokla or Thepla.",
    "jharkhand": "We can recommend some local favorites from Jharkhand, like Litti Chokha or Dhuska.",
    "karnataka": "We recommend some classic Karnataka dishes, like Bisi Bele Bath or Mysore Dosa.",
    "kerala": "We detect a Kerala accent. You might enjoy our Appam, Puttu, or Avial!",
    "tamil": "How about some delicious Tamil food? We recommend Chettinad Chicken or Pongal."
}

# --- 3. The Main Prediction Function ---
def predict_and_recommend(audio_input):
    """
    Takes audio input from Gradio, predicts accent, and returns a recommendation.
    Gradio provides the audio as a (sample_rate, waveform) tuple.
    """
    try:
        # 1. Get audio from Gradio input
        # Check if input is None (e.g., user just clicked the 'X' on the audio)
        if audio_input is None:
            return "Please record or upload audio to get a recommendation."

        sr, waveform = audio_input

        # 2. Ensure it's mono and has signal
        if waveform.ndim > 1:
            waveform = np.mean(waveform, axis=1)
        if np.sum(np.abs(waveform)) == 0:
            return "No audio detected. Please record or upload a valid file."

        # 3. Extract MFCCs (same settings as our training)
        mfccs = librosa.feature.mfcc(y=waveform.astype(float), sr=sr, n_mfcc=20)
        mfccs_mean = np.mean(mfccs, axis=1)

        # 4. Scale Features (CRITICAL STEP)
        mfccs_scaled = scaler.transform(mfccs_mean.reshape(1, -1))

        # 5. Make Prediction
        prediction_int = model.predict(mfccs_scaled)[0]
        prediction_proba = model.predict_proba(mfccs_scaled)
        confidence = np.max(prediction_proba) * 100

        # 6. Get Recommendation
        accent_name = int_to_label[prediction_int]
        recommendation = recommendation_map.get(accent_name, "No recommendation available.")

        # 7. Format the output string
        output_text = f"--- 🍲 Accent-Aware Recommendation ---\n\n"
        output_text += f"Predicted Accent: {accent_name.replace('_', ' ').title()}\n"
        output_text += f"Confidence: {confidence:.2f}%\n\n"
        output_text += f"Recommendation:\n> {recommendation}"

        return output_text

    except Exception as e:
        return f"An error occurred: {e}"

# --- 4. Launch the Gradio Interface (Side-by-Side Layout) ---

# We use gr.Blocks() for more control over the layout.
with gr.Blocks(theme=gr.themes.Soft()) as iface:
    gr.Markdown("# Accent-Aware Cuisine Recommendation System")
    gr.Markdown("This app analyzes your English accent to detect your native Indian language and recommend cuisine from your region.")

    with gr.Row():
        # --- Column 1: Input ---
        with gr.Column(scale=1):

            # --- THE FIX IS HERE ---
            # We've added "upload" to the sources list.
            audio_in = gr.Audio(sources=["microphone", "upload"], type="numpy", label="Record or Upload Audio")

        # --- Column 2: Output ---
        with gr.Column(scale=2):
            text_out = gr.Text(label="Recommendation", lines=6)

    # Connect the input to the function to the output
    # 'change' runs when a new recording or upload is submitted
    audio_in.change(fn=predict_and_recommend, inputs=audio_in, outputs=text_out)

# share=True creates a public, shareable link
iface.launch(share=True, debug=True)